Fire research at the National Institute of Standards and Technology (NIST) dates back to the early 1900's following the need for a standardized fire hydrant coupling after the Great Baltimore Fire. Fast-forward to the 1980s and for the last 30 years, NIST researchers have been performaning experiments and running computer simulations following fire incidents to advance the field of fire safety.

NIST researchers wrote detailed reports of their work following these fires, but an intuitive, centralized source of this material did not exist. Therefore, we wanted to build an interface to fix this. We felt a map that showed where the fires occured with interactivity to link to the reports would be an interesting project.

In [11]:
import pandas as pd
from bokeh.models.glyphs import Circle
from bokeh.plotting import show, output_notebook,figure
from bokeh.models import (
    GMapPlot, GMapOptions, Range1d, ColumnDataSource, LinearAxis,
    PanTool, WheelZoomTool,HoverTool, TapTool, OpenURL)
output_notebook()

BokehJS successfully loaded.

# First, let's load and take a look at the data file of fires.

In [12]:
data = pd.read_csv('study_data.csv')

In [13]:
data.head()

,Study,Latitude,Longitude,Color,Type,Date,Report
0,Simulation of an Attic Fire in a Wood Frame Re...,41.802088,-87.681981,Navy,LODD/LODI,2-Nov-12,http://dx.doi.org/10.6028/NIST.TN.1838
1,Simulation of a Fire in a Hillside Residential...,37.739505,-122.439223,Navy,LODD/LODI,2-Jun-11,http://dx.doi.org/10.6028/NIST.TN.1856
2,Simulation of a Residential Wind Driven Baseme...,38.965891,-76.917754,Navy,LODD/LODI,24-Feb-12,http://dx.doi.org/10.6028/NIST.TN.1870
3,Simulation of the Dynamics of a Wind-Driven Fi...,29.679490,-95.282329,Navy,LODD/LODI,12-Apr-09,http://www.nist.gov/customcf/get_pdf.cfm?pub_i...
4,Simulation of the Dynamics of a Fire in a Two-...,40.404689,-91.382416,Red,LODD/LODI & Civilian Loss,22-Dec-99,http://www.nist.gov/customcf/get_pdf.cfm?pub_i...


The addresses of the fires were geocoded to generate the lattitude and longitude for each fire. The studies were then coded into one of four categories: <span style="color:blue">LODD/LODI</span> (a fire with a firefighter Line of Duty Death or Line of Duty Injury), <span style="color:green">Civilian Loss</span>, <span style="color:orange">WUI</span> (wildland urban interface fire) or <span style="color:red">LODD/LODI and Civilian Loss</span>. The data file also includes the NIST hosting link of the report so that we can link to it through first class bokeh features.

___

# The next step is to load in a simple map.
The latitude, longitude, and zoom are set to view all of the United States and Puerto Rico.  

In [14]:
x_range = Range1d()
y_range = Range1d()

map_options = GMapOptions(lat=39., lng=-98, zoom=3)

plot = GMapPlot(
    x_range=x_range, y_range=y_range,
    map_options=map_options,
    title = "NIST Fire Studies", plot_width=875, plot_height=500
)
plot.map_options.map_type="terrain"

# Adding Interaction

We wanted the the study name, type, and date of the fire to be available upon hover. Second, we wanted the glyphs to be clickable to direct users to the full study reports (the main purpose of this exercise).

In [15]:
source = ColumnDataSource({'lat':data['Latitude'],'lon':data['Longitude'],'studys': data['Study'],
                           'report': data['Report'],'fill':data['Color'],'type':data['Type'],'date':data['Date']})
circle = Circle(x="lon",y="lat",size=15,fill_color="fill")
plot.add_glyph(source, circle)

pan = PanTool()
wheel_zoom = WheelZoomTool()
hover = HoverTool()
hover.tooltips = [('Study Title','@studys'),('Date','@date'),('Type','@type')]
tap = TapTool()
url = "@report"
TapTool.callback = OpenURL(url=url)

plot.add_tools(pan,wheel_zoom,hover,tap)

In [16]:
show(plot)

This is clearly a simple example, but that is point. With minimal code, the final product is an interative map that highlights where prior fires have been studied by NIST researchers.